# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846716669405                   -0.70    5.0
  2   -7.852320096770       -2.25       -1.53    1.0
  3   -7.852646023759       -3.49       -2.53    3.0
  4   -7.852646677365       -6.18       -3.41    2.2
  5   -7.852646686217       -8.05       -4.67    1.8
  6   -7.852646686725       -9.29       -5.62    2.5
  7   -7.852646686730      -11.32       -6.62    3.2
  8   -7.852646686730      -14.05       -7.36    1.5
  9   -7.852646686730   +  -14.45       -8.29    2.0
 10   -7.852646686730      -14.45       -9.26    4.0
 11   -7.852646686730      -14.75       -9.81    1.0
 12   -7.852646686730   +  -14.75      -10.87    2.0
 13   -7.852646686730      -14.75       -9.81    1.0
 14   -7.852646686730   +  -14.45       -9.80    1.0
 15   -7.852646686730      -14.45      -10.16    2.0
 16   -7.852646686730   +  -14.75       -9.63    1.0
 17   -7.852646686730   +    -Inf      -10.42 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846896200289                   -0.70           4.5
  2   -7.852528706995       -2.25       -1.63   0.80    2.0
  3   -7.852637515773       -3.96       -2.71   0.80    1.0
  4   -7.852646480728       -5.05       -3.30   0.80    2.0
  5   -7.852646669805       -6.72       -4.14   0.80    1.0
  6   -7.852646686372       -7.78       -4.72   0.80    2.0
  7   -7.852646686721       -9.46       -5.47   0.80    1.8
  8   -7.852646686729      -11.07       -6.65   0.80    1.8
  9   -7.852646686730      -12.44       -7.85   0.80    2.5
 10   -7.852646686730      -15.05       -8.32   0.80    2.5
 11   -7.852646686730   +    -Inf       -9.27   0.80    1.5
 12   -7.852646686730      -15.05       -9.78   0.80    2.5
 13   -7.852646686730   +  -14.75      -10.57   0.80    1.0
 14   -7.852646686730   +  -15.05      -11.45   0.80    2.2
 15   -7.852646686730   +  -15.05      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.395547e+01     3.540831e+00
 * time: 0.5083088874816895
     1     1.415993e+00     1.789928e+00
 * time: 0.7569029331207275
     2    -1.950175e+00     1.912752e+00
 * time: 0.7854628562927246
     3    -3.704713e+00     1.721673e+00
 * time: 0.826042890548706
     4    -5.176779e+00     1.436891e+00
 * time: 0.8682630062103271
     5    -6.705521e+00     1.149900e+00
 * time: 0.9111578464508057
     6    -7.428553e+00     7.441189e-01
 * time: 0.9591808319091797
     7    -7.645872e+00     6.446570e-01
 * time: 0.9911179542541504
     8    -7.764543e+00     1.247534e-01
 * time: 1.0202958583831787
     9    -7.802409e+00     1.380080e-01
 * time: 1.0492219924926758
    10    -7.827375e+00     9.568594e-02
 * time: 1.0782089233398438
    11    -7.839580e+00     5.222852e-02
 * time: 1.107990026473999
    12    -7.848995e+00     2.524409e-02
 * time: 1.137826919555664
    13    -7.851602e+00     2.050782e-02
 * time: 1.1671550273895

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846846546711                   -0.70    4.5
  2   -7.852315972563       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686709                   -2.54
  2   -7.852646686730      -10.69       -5.97
  3   -7.852646686730   +    -Inf      -12.74


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.1452877041438066e-13
|ρ_newton - ρ_scfv| = 3.0055759633080983e-13
|ρ_newton - ρ_dm|   = 7.10130475334042e-10
